In [1]:
#|hide
#|default_exp export

# export
> Exporting a notebook to a library

In [2]:
#|export
from nbprocess.read import *
from nbprocess.maker import *
from nbprocess.imports import *
from nbprocess.process import *

from fastcore.script import *
from fastcore.basics import *
from fastcore.imports import *

from collections import defaultdict

In [3]:
#|hide
from fastcore.test import *
from pdb import set_trace
from importlib import reload
from fastcore import shutil

In [4]:
#|export
class ExportModuleProc:
    "A processor which exports code to a module"
    def __init__(self): self.modules,self.in_all = defaultdict(L),defaultdict(L)
    def _default_exp_(self, nbp, cell, exp_to): self.default_exp = exp_to
    def _exporti_(self, nbp, cell, exp_to=None): self.modules[ifnone(exp_to, '#')].append(nbp.cell)
    def _export_(self, nbp, cell, exp_to=None):
        self._exporti_(nbp, cell, exp_to)
        self.in_all[ifnone(exp_to, '#')].append(nbp.cell)

Specify `dest` where the module(s) will be exported to, and optionally a class to use to create the module (`ModuleMaker`, by default).

Exported cells are stored in a `dict` called `modules`, where the keys are the modules exported to. Those without an explicit module are stored in the `'#'` key, which will be exported to `default_exp`.

In [5]:
everything_fn = '../tests/01_everything.ipynb'

exp = ExportModuleProc()
proc = NBProcessor(everything_fn, exp)
proc.process()
test_eq(exp.default_exp, 'everything')
assert 'print_function'  in exp.modules['#'][0].source
assert 'h_n' in exp.in_all['some.thing'][0].source

In [6]:
#|export
def create_modules(path, dest, procs=None, debug=False, mod_maker=ModuleMaker):
    "Create module(s) from notebook"
    exp = ExportModuleProc()
    nb = NBProcessor(path, [exp]+L(procs), debug=debug)
    nb.process()
    for mod,cells in exp.modules.items():
        all_cells = exp.in_all[mod]
        name = exp.default_exp if mod=='#' else mod
        mm = mod_maker(dest=dest, name=name, nb_path=path, is_new=mod=='#')
        mm.make(cells, all_cells)

Let's check we can import a test file:

In [7]:
#|eval: false
shutil.rmtree('tmp', ignore_errors=True)
create_modules('../tests/00_some.thing.ipynb', 'tmp')

g = exec_new('import tmp.some.thing')
test_eq(g['tmp'].some.thing.__all__, ['a'])
test_eq(g['tmp'].some.thing.a, 1)

We'll also check that our 'everything' file exports correctly:

In [8]:
#|eval: false
create_modules(everything_fn, 'tmp')

g = exec_new('import tmp.everything; from tmp.everything import *')
_alls = L("a b d e m n o p q".split())
for s in _alls.map("{}_y"): assert s in g, s
for s in "c_y_nall _f_y_nall g_n h_n i_n j_n k_n l_n".split(): assert s not in g, s
for s in _alls.map("{}_y") + ["c_y_nall", "_f_y_nall"]: assert hasattr(g['tmp'].everything,s), s

That notebook should also export one extra function to `tmp.some.thing`:

In [9]:
#|eval: false
del(sys.modules['tmp.some.thing']) # remove from module cache
g = exec_new('import tmp.some.thing')
test_eq(g['tmp'].some.thing.__all__, ['a','h_n'])
test_eq(g['tmp'].some.thing.h_n(), None)

In [10]:
#|export
def nb_export(nbname, lib_path=None):
    if lib_path is None: lib_path = get_config().path('lib_path')
    create_modules(nbname, lib_path)

## Export -

In [11]:
#|eval: false
Path('../nbprocess/export.py').unlink(missing_ok=True)
nb_export('04a_export.ipynb')

g = exec_new('import nbprocess.export')
assert hasattr(g['nbprocess'].export, 'nb_export')